In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import gc

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import joblib
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = []
list_dir = os.listdir('./wiki_data')
list_dir = sorted(list_dir)
for path in list_dir:
    if 'wiki_2023_index' in path or 'my' in path or 'debert' in path or 'ipynb' in path:
        continue
    print(path)
    x = pd.read_parquet(os.path.join('./wiki_data', path))
    x['file'] = path
    df.append(x)
df = pd.concat(df, axis=0).reset_index(drop=True)
df['id'] = df['id'].astype(int)

display(df)

a.parquet
b.parquet
c.parquet
d.parquet
e.parquet
f.parquet
g.parquet
h.parquet
i.parquet
j.parquet
k.parquet
l.parquet
m.parquet
n.parquet
number.parquet
o.parquet
other.parquet
p.parquet
q.parquet
r.parquet
s.parquet
t.parquet
u.parquet
v.parquet
w.parquet
x.parquet
y.parquet
z.parquet


,id,title,text,categories,file
0,49495844,A & B High Performance Firearms,A & B High Performance Firearms was a competit...,"[Defunct firearms manufacturers, Defunct manuf...",a.parquet
1,3579086,A & C Black,A & C Black is a British book publishing compa...,"[Encyclopædia Britannica, Ornithological publi...",a.parquet
2,62397582,A & F Harvey Brothers,"A & F Harvey Brothers, first Spinning Cotton M...",[Cotton mills],a.parquet
3,15547032,A & G Price,A & G Price Limited is an engineering firm and...,"[Locomotive manufacturers of New Zealand, Tham...",a.parquet
4,8021609,A & M Karagheusian,thumb|right|238px|A portion of the Karagheusia...,"[1904 establishments in the United States, Arm...",a.parquet
...,...,...,...,...,...
6286770,18881188,Zəyəm Cırdaxan,"Zəyəm Cırdaxan (also Dzegam-Dzhirdakhan, Dzega...",[Populated places in Shamkir District],z.parquet
6286771,18873112,"Zəyəm, Qakh","Zəyəm (also, Zagam, Zagyam, and Zeyam; ) is a ...",[Populated places in Qakh District],z.parquet
6286772,18881204,"Zəyəm, Shamkir","Zəyəm (also, Dzagam, Dzegam, Dzegan, Guseynbei...",[Populated places in Shamkir District],z.parquet
6286773,18920475,"Zəyəm, Zaqatala","Zəyəm (also, Zagam and Zagyam) is a village an...",[Populated places in Zaqatala District],z.parquet


In [3]:
index2id = df.to_dict()['id']
train = pd.read_parquet('./data/recall_val_Top1000.parquet')

def func(x):
    return [index2id[i] for i in x]

train['Top1000'] = train['Top1000'].apply(func)

train

,id,prompt,A,B,C,D,E,answer,wiki_text,page_id,page_title,stem_label,prompt_answer,Top1000
0,47000,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are some adverse effects associated with ...,"[73305281, 37431, 47760832, 3708060, 3169057, ..."
1,47001,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What is the goal of the UN sustainability deve...,"[73305281, 2740944, 3169057, 37431, 31109011, ..."
2,47002,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are green solvents derived from? Processi...,"[73305281, 37431, 5048079, 936085, 3527488, 27..."
3,47003,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What makes it difficult to classify green solv...,"[73305281, 936085, 3527488, 52791592, 37431, 2..."
4,47004,What is the significance of the Nora Stone?,It is the oldest Phoenician inscription found ...,It is the only Phoenician inscription found in...,It is the first evidence of Phoenician coloniz...,It is the longest Phoenician inscription ever ...,It is the only known inscription honoring a go...,A,The Nora Stone or Nora Inscription is an ancie...,23809512,Nora Stone,T,What is the significance of the Nora Stone? It...,"[23809512, 67592869, 71513100, 65682027, 65575..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2279,49279,What is the purpose of acoustic cleaning?,To remove material from surfaces,To generate powerful sound waves,To handle bulk granular materials,To reduce the need for manual cleaning,To build material-handling equipment,A,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,What is the purpose of acoustic cleaning? To r...,"[4117074, 3020263, 8414306, 3636036, 59773453,..."
2280,49280,How does an acoustic cleaner work?,By generating powerful sound waves,By shaking particulates loose from surfaces,By removing the buildup of material on surfaces,By reducing the need for manual cleaning,By handling bulk granular materials,B,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,How does an acoustic cleaner work? By generati...,"[4117074, 3020263, 8414306, 14132438, 14471620..."
2281,49281,What is an acoustic cleaner made of?,Compressed air,Electricity,A diaphragm,Stainless steel,Titanium,D,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,What is an acoustic cleaner made of? Compresse...,"[4117074, 3020263, 8414306, 14471620, 14132438..."
2282,49282,Why is compressed air used to power an acousti...,To generate powerful sound waves,To shake particulates loose from surfaces,To ensure performance and longevity,To reduce the need for manual cleaning,To avoid sparking and explosions,E,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,Why is compressed air used to power an acousti...,"[56765008, 381774, 17181237, 4117074, 72612627..."


In [4]:
page_ids = list(df['id'].unique())
train_filted = train[train['page_id'].isin(page_ids)].reset_index(drop=True)
train_filted

,id,prompt,A,B,C,D,E,answer,wiki_text,page_id,page_title,stem_label,prompt_answer,Top1000
0,47000,What are some adverse effects associated with ...,Burns and eczema,Cancer and brain disorder,Fetotoxicity and inflammations of peripheral n...,All of the above,None of the above,D,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are some adverse effects associated with ...,"[73305281, 37431, 47760832, 3708060, 3169057, ..."
1,47001,What is the goal of the UN sustainability deve...,To promote the use of petrochemical solvents,To develop green and sustainable solvents,To increase the use of nonpolar solvents,To classify solvents into polar and nonpolar c...,To decrease the use of solvents in various dom...,B,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What is the goal of the UN sustainability deve...,"[73305281, 2740944, 3169057, 37431, 31109011, ..."
2,47002,What are green solvents derived from?,Processing of agricultural crops,Petrochemical sources,Regular solvents,Chemical reactions,Water and air pollution,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What are green solvents derived from? Processi...,"[73305281, 37431, 5048079, 936085, 3527488, 27..."
3,47003,What makes it difficult to classify green solv...,Their chemical structure and source can differ,They have adverse effects on health and safety,They are associated with water and air pollution,They are derived from petrochemical sources,They are nonpolar solvents,A,A solvent is a substance that is liquid at the...,73305281,Green solvent,M,What makes it difficult to classify green solv...,"[73305281, 936085, 3527488, 52791592, 37431, 2..."
4,47004,What is the significance of the Nora Stone?,It is the oldest Phoenician inscription found ...,It is the only Phoenician inscription found in...,It is the first evidence of Phoenician coloniz...,It is the longest Phoenician inscription ever ...,It is the only known inscription honoring a go...,A,The Nora Stone or Nora Inscription is an ancie...,23809512,Nora Stone,T,What is the significance of the Nora Stone? It...,"[23809512, 67592869, 71513100, 65682027, 65575..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,49279,What is the purpose of acoustic cleaning?,To remove material from surfaces,To generate powerful sound waves,To handle bulk granular materials,To reduce the need for manual cleaning,To build material-handling equipment,A,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,What is the purpose of acoustic cleaning? To r...,"[4117074, 3020263, 8414306, 3636036, 59773453,..."
2125,49280,How does an acoustic cleaner work?,By generating powerful sound waves,By shaking particulates loose from surfaces,By removing the buildup of material on surfaces,By reducing the need for manual cleaning,By handling bulk granular materials,B,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,How does an acoustic cleaner work? By generati...,"[4117074, 3020263, 8414306, 14132438, 14471620..."
2126,49281,What is an acoustic cleaner made of?,Compressed air,Electricity,A diaphragm,Stainless steel,Titanium,D,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,What is an acoustic cleaner made of? Compresse...,"[4117074, 3020263, 8414306, 14471620, 14132438..."
2127,49282,Why is compressed air used to power an acousti...,To generate powerful sound waves,To shake particulates loose from surfaces,To ensure performance and longevity,To reduce the need for manual cleaning,To avoid sparking and explosions,E,Acoustic cleaning is a maintenance method used...,4117074,Acoustic cleaning,E,Why is compressed air used to power an acousti...,"[56765008, 381774, 17181237, 4117074, 72612627..."


In [5]:
cnt = 0
for _, row in tqdm(train_filted.iterrows(), total=len(train_filted)):
    if row['page_id'] in row['Top1000']:
        cnt += 1
print(cnt, cnt/len(train_filted))

100%|██████████| 2129/2129 [00:00<00:00, 21139.57it/s]

2044 0.960075152653828


In [6]:
df['categories'] = df['categories'].apply(lambda x: ' '.join(x))
df['full_text'] = df['title'] + ' ' + df['categories'] + ' ' + df['text']
del df['title'], df['text'], df['categories'], df['file']
gc.collect()

0

In [ ]:
%%time

vectorizer = TfidfVectorizer(max_df=0.9, 
                             min_df=0.1, 
                             max_features=200000, 
                             stop_words='english',
                             use_idf=True,
                             ngram_range=(1,3))

vectorizer.fit(df['full_text'].values)

In [10]:
joblib.dump(vectorizer, 'tfidf_model.joblib')

['tfidf_model.joblib']

In [11]:
candidate_texts = df.set_index('id').to_dict()['full_text']

In [ ]:
cnt = 0
for _, row in tqdm(train_filted.iterrows(), total=len(train_filted)):
    top_ids = row['Top1000']
    search_text = row['prompt_answer']
    target = row['page_id']
    candidates = [candidate_texts[i] for i in top_ids]
    candidate_vectors = vectorizer.transform(candidates)
    knn = NearestNeighbors(n_neighbors=10, metric='cosine')
    knn.fit(candidate_vectors)
    vectors = vectorizer.transform([search_text])
    distances, indices = knn.kneighbors(vectors, n_neighbors=10)
    candidate_page_id = [top_ids[i] for i in indices[0]]
    if target in candidate_page_id:
        cnt += 1

  0%|          | 0/2129 [00:00<?, ?it/s]

In [44]:
print(cnt, cnt/len(train_filted))

481 0.22592766557069047
